In [1]:
import pandas as pd
import numpy as np
from numba import njit

## Maximum-likelihood estimation

In [2]:
class MLestimation:
    # Input: Sequence of triplets (i.e., NM) consisting of time (t), total active nodes (N) and total edges (M).
    def __init__(self, NM):
        self.NM = NM
        self.PARAMS = parcombs2
        self.LKFNS  = Likelihoodfns
    
    @staticmethod
    @njit(nopython=True)  
    def prob_func(A,E,likelihood):
        probindex = np.where((likelihood[0:,0]==A) & (likelihood[0:,1]==E))
        if probindex[0].size!=0:
            probability = likelihood[probindex][0,-1]
        return probability
    
    def MLE(self):
        """
        Input.

        NMseq: Sequence of triplets consisting of time (t), total active nodes (N) and total edges (M).

        Output. 
        Estimates of overall act and pop size: kappa and Np. Type: (pandas.DataFrame)
        """
        
        knp_ests = []
        for elem in self.NM:
            N,M = elem
            prob_NM = []
            combid = []
            for fid,LF in self.LKFNS.items():
                prob_NM.append(self.prob_func(N,M,LF))
                combid.append(fid)
            estparam_index = np.argmax(np.asarray(prob_NM))
            estparams_id = combid[estparam_index]
            knp_ests.append( estparams_id)
            
        estimates = pd.DataFrame({"params":knp_ests})
        kappavalues, npvalues = self.estimators(estimates)
        estimates["kappa"] = kappavalues
        estimates["np"] = npvalues
        return estimates

    @staticmethod
    def estimators(estimates):
        PARAMS = parcombs2
        """
        Input: Helper that takes sequences of IDs for (kappa, Np) combinations which have been
        identified as estimators by the MLE instance above.

        Output: DataFrame of estimates kappa_hat and Np_hat given a time window t

        The function matches the ids of the parameter combination (i.e, PARAMS) with the ML
        estimators (kappa_hat and Np_hat).
        """
        kappa_seq = []
        np_seq = []
        for combs in estimates["params"]:
            kappa,np = PARAMS[combs]
            kappa_seq.append(kappa)
            np_seq.append(np)
        return kappa_seq,np_seq
 
    

/opt/anaconda3/lib/python3.7/site-packages/numba/core/decorators.py:255: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)
